# Домашнее задание 2 по обработке текстов

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная обработка данных

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

##  Часть 2. Базовый метод классификации

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:
from nltk.util import ngrams

In [160]:
import numpy as np
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
import string
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
DATA_DIR='names_data'

In [195]:
females = read_file('female.txt')
males   = read_file('male.txt')
females_uniq = np.setdiff1d(females, males)
males_uniq = np.setdiff1d(males, females)

In [105]:
cv = CountVectorizer(analyzer='word')
cv.fit(t)
cv.vocabulary_

{'abcd': 0, 'bcde': 1, 'cdef': 2, 'erty': 3, 'qwer': 4, 'wert': 5}

In [109]:
cv.transform(t).todense()

matrix([[1, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 1]])

In [253]:
TRAIN_SIZE = 2100
TEST_SIZE = 440
NGRAM_SIZE = 2

In [254]:
female_train, female_test = train_test_split(females_uniq, train_size = TRAIN_SIZE, test_size = TEST_SIZE)
male_train, male_test = train_test_split(males_uniq, train_size = TRAIN_SIZE, test_size = TEST_SIZE)

In [255]:
X_train_names = np.concatenate((female_train, male_train))
X_test_names = np.concatenate((female_test, male_test))

In [256]:
y_train = np.concatenate((np.zeros(TRAIN_SIZE),np.ones(TRAIN_SIZE))) 
y_test = np.concatenate((np.zeros(TEST_SIZE), np.ones(TEST_SIZE)))

In [257]:
X_train_ngrams = get_ngrams_list(X_train_names,NGRAM_SIZE)
X_test_ngrams = get_ngrams_list(X_test_names, NGRAM_SIZE)

In [258]:
cv = CountVectorizer(analyzer='word')
cv.fit(X_train_ngrams)
X_train = cv.transform(X_train_ngrams)
X_test = cv.transform(X_test_ngrams)

In [259]:
X_train.shape

(4200, 425)

In [260]:
MNB_clf = MultinomialNB(alpha=1)
MNB_clf.fit(X_train, y_train)
train_predicted = MNB_clf.predict(X_train)
print("TRAIN_Scores")
print_scores(train_predicted, y_train)
print("TEST_Scores")
print_scores(MNB_clf.predict(X_test), y_test)

TRAIN_Scores
acc=0.7748
micro F1=0.7748, micro P=0.7748, micro R=0.0953, micro RC=0.7748
macro F1=0.7745, macro P=0.7748, macro R=0.0953, macro RC=0.7759 

TEST_Scores
acc=0.7375
micro F1=0.7375, micro P=0.7375, micro R=-0.0500, micro RC=0.7375
macro F1=0.7375, macro P=0.7375, macro R=-0.0500, macro RC=0.7375 



# Library

In [159]:
def print_scores(pred, true):
    acc = accuracy_score(pred,true)
    acc = accuracy_score(pred, true)
    micro_f1 = f1_score(pred, true, average = 'micro')
    micro_p =  precision_score(pred, true, average = 'micro')
    micro_rc =  recall_score(pred, true, average = 'micro' )
    micro_r =  r2_score(pred, true)
    macro_f1 = f1_score(pred, true, average = 'macro')
    macro_p =  precision_score(pred, true, average = 'macro')
    macro_r =  r2_score(pred, true)
    macro_rc = recall_score(pred, true, average = 'macro' )
    print('acc={0:1.4f}'.format(acc))
    print('micro F1={0:1.4f}, micro P={1:1.4f}, micro R={2:1.4f}, micro RC={3:1.4f}'.format(micro_f1, micro_p, micro_r, micro_rc))
    print('macro F1={0:1.4f}, macro P={1:1.4f}, macro R={2:1.4f}, macro RC={3:1.4f} \n'.format(macro_f1, macro_p, macro_r, macro_rc))

In [114]:
def read_file(fn:str):
    """
        Прочитать файл и вернуть np.array(dtype=np.str())
    """
    f = open(DATA_DIR + '/' + fn, 'r')
    t = []
    for i in f:
        i = i.strip()
        t.append(i)
    rv = np.array(t, dtype=np.str)
    return rv

In [115]:
def get_ngrams_list(l: list, n: int):
    a = []
    for i in l:
        t = ngrams(i,n)
        t2 = ''
        for j in t:
            t2 += ''.join((map(str,j))) + ' '
        a.append(t2)
    return a

In [188]:
def remove_dup_names(a: list, b:list):
    for i in a:
        if i in b:
            a.remove(i)
            b.remove(i)

# Flood.

In [120]:
t = get_ngrams_list(['abcdef', 'qwerty'],4)
cv = CountVectorizer(analyzer='word')
cv.fit(t)
cv.vocabulary_

{'abcd': 0, 'bcde': 1, 'cdef': 2, 'erty': 3, 'qwer': 4, 'wert': 5}

In [191]:
t = np.array(['a','b','c','d'])
np.delete(t['a'])
t

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices